# Answering Business Questions Using SQL

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Selecting Ablums to Purchase

In [64]:
%%sql
WITH usa_tracks_sold AS 
    (
        SELECT 
            il.*
              FROM invoice_line il
            INNER JOIN invoice i on il.invoice_id=i.invoice_id
            INNER JOIN customer c on i.customer_id=c.customer_id
            WHERE c.country='USA'
    )
    
SELECT 
    g.name genre,
    COUNT(uts.invoice_line_id) sold_tot,
    ROUND(CAST(COUNT(uts.track_id) AS Float) / (SELECT COUNT(*) FROM usa_tracks_sold), 2) sold_per
      FROM usa_tracks_sold uts
    LEFT JOIN track t on t.track_id = uts.track_id
    LEFT JOIN genre g ON t.genre_id=g.genre_id
    GROUP BY t.genre_id
    ORDER BY sold_tot DESC


 * sqlite:///chinook.db
Done.


genre,sold_tot,sold_per
Rock,561,0.53
Alternative & Punk,130,0.12
Metal,124,0.12
R&B/Soul,53,0.05
Blues,36,0.03
Alternative,35,0.03
Pop,22,0.02
Latin,22,0.02
Hip Hop/Rap,20,0.02
Jazz,14,0.01


## Analyzing Employee Sales Performance

In [83]:
%%sql
WITH customer_expenses AS 
    (
        SELECT 
             c.*,
             SUM(i.total) total
          FROM customer c
         INNER JOIN invoice i ON c.customer_id=i.customer_id
         GROUP BY c.customer_id
    )
    
SELECT
     e.first_name || ' ' || e.last_name employee_name,
     e.title,
     e.hire_date,
     SUM(c.total) total_sales
  FROM customer_expenses c
 LEFT JOIN employee e ON e.employee_id=c.support_rep_id
 GROUP BY e.employee_id
 ORDER BY total_sales DESC

 * sqlite:///chinook.db
Done.


employee_name,title,hire_date,total_sales
Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,1731.5099999999998
Margaret Park,Sales Support Agent,2017-05-03 00:00:00,1584.0000000000002
Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,1393.92


## Analyzing Sales by Country